<a href="https://colab.research.google.com/github/nachospimp/pf/blob/main/02_market_dataclase12_act.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Mar 21 19:31:33 2024

@author: Usuario
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import importlib
import os

# import or own files and reload
import random_variables1
importlib.reload(random_variables1)
import market_dataclase12
importlib. reload(market_dataclase12)


#  inputs
directory = '/Users/giovannamontalvo/2024/Data1/' #Harcoded
ric = 'ALFAA.MX'

#CEMEXCPO.MX, ALSEA.MX, BIMBOA.MX, GMEXICOB.MX, FEMSAUBD.MX, FIBRAPL14.MX,

# computations
dist = market_dataclase12.distribution(ric)
dist.load_timeseries()
dist.plot_timeseries()
dist.compute_stats()
dist.adf_test()  # Ejecuta la prueba ADF
dist.plot_histrogram()

# Ajustar y prever con ARIMA
dist.fit_arima(p=1, q=1)  # Puedes ajustar los valores de p y q según sea necesario
forecast = dist.forecast_arima(steps=10)
dist.plot_forecast(steps=10)
# Análisis de residuos
dist.analyze_residuals()

# loop to check normality in real distributions
rics = []
is_normals = []
is_stationaries = []
for file_name in os.listdir(directory):
    print('file_name = ' + file_name)
    ric = file_name.split('.')[0]
    if ric== 'ReadMe':
        continue
    #compute stats
    dist = market_dataclase12.distribution(ric)
    dist.load_timeseries()
    dist.compute_stats()
    dist.adf_test()  # Agregar prueba ADF
    # generate list
    rics.append(ric)
    is_normals.append(dist.is_normal)
    is_stationaries.append(dist.is_stationary)
df = pd.DataFrame()
df['ric'] = rics
df['is_normal'] = is_normals
df['is_stationary'] = is_stationaries
df = df.sort_values(by='is_normal', ascending=False)
df = df.sort_values(by='is_stationary', ascending=False)



In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 26 13:09:18 2024

@author: Usuario
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import importlib
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.stats import shapiro


def load_timeseries(ric):
    directory = '/Users/giovannamontalvo/2024/Data1/' #Harcoded
    path = directory + ric + '.csv'
    raw_data = pd.read_csv(path)
    t = pd.DataFrame()
    t['date']= pd.to_datetime(raw_data['Date'], yearfirst=True) # dayfirst=True (some cases, check data!)
    t['close'] = raw_data['Close']
    t = t.sort_values(by='date', ascending=True)
    t['close_previous'] = t['close'].shift(1)
    t['return'] = t['close']/t['close_previous'] - 1
    t = t.dropna()
    t = t.reset_index (drop=True)
    return t

def synchronise_timeseries(benchmark,security):
    timeseries_x = load_timeseries(benchmark)
    timeseries_y = load_timeseries(security)
    timestamps_x = list(timeseries_x['date'].values)
    timestamps_y = list(timeseries_y['date'].values)
    timestamps = list(set(timestamps_x) & set(timestamps_y))
    timeseries_x = timeseries_x[timeseries_x['date'].isin(timestamps)]
    timeseries_x = timeseries_x.sort_values(by='date', ascending=True)
    timeseries_x = timeseries_x.reset_index(drop=True)
    timeseries_y = timeseries_y[timeseries_y['date'].isin(timestamps)]
    timeseries_y = timeseries_y.sort_values(by='date', ascending=True)
    timeseries_y = timeseries_y.reset_index(drop=True)
    timeseries = pd.DataFrame()
    timeseries['date'] = timeseries_x['date']
    timeseries['close_x'] = timeseries_x['close']
    timeseries['close_y'] = timeseries_y['close']
    timeseries['return_x'] = timeseries_x['return']
    timeseries['return_y'] = timeseries_y['return']
    return timeseries

def synchronise_returns(rics):
    df = pd.DataFrame()
    dic_timeseries = {}
    timestamps = []
    for ric in rics:
        t = load_timeseries(ric)
        dic_timeseries[ric] = t
        if len(timestamps) == 0:
            timestamps = list(t['date'].values)
        temp_timestamps = list(t['date'].values)
        timestamps = list(set(timestamps) & set(temp_timestamps))
    for ric in dic_timeseries:
        t = dic_timeseries[ric]
        t =  t[t['date'].isin(timestamps)]
        t = t.sort_values(by='date', ascending=True)
        t = t.dropna()
        t = t.reset_index(drop=True)
        dic_timeseries[ric] = t
        if df.shape[1] == 0:
            df['date'] = timestamps
        df[ric] = t['return']
    return df


class distribution:

    # constructor
    def __init__(self, ric, decimals= 5):
        self.ric = ric
        self.decimals = 5
        self.str_tittle = None
        self.timeseries = None
        self.vector = None
        self.mean_annual = None
        self.volatility_annual = None
        self.sharpe_ratio = None
        self.var_95 = None
        self.skewness = None
        self.kurtosis = None
        self.jb_stat = None
        self.p_value = None
        self.is_normal = None
        self.adf_stat = None
        self.adf_p_value = None
        self.is_stationary = None
        self.model_fit = None
        self.forecast = None


    def load_timeseries(self):
        self.timeseries = load_timeseries(self.ric)
        self.vector = self.timeseries['return'].values
        self.size = len(self.vector)
        self.str_tittle = self.ric + " | real data"

        # Prueba ADF para estacionariedad
    def adf_test(self):
        adf_result = adfuller(self.vector)
        self.adf_stat = adf_result[0]
        self.adf_p_value = adf_result[1]
        self.is_stationary = self.adf_p_value < 0.05  # True si la serie es estacionaria

    def plot_timeseries(self):
        plt.figure()
        self.timeseries.plot(kind='line', x='date', y='close', grid=True, color='blue',\
               title='Time series of close prices for'+self.ric)
        plt.show()

    def compute_stats (self, factor = 252):
        self.mean_annual = st.tmean(self.vector) * 252
        self.volatility_annual = st.tstd(self.vector) * np.sqrt(factor)
        self.sharpe_ratio = self.mean_annual / self.volatility_annual if self.volatility_annual > 0 else 0.0
        self.var_95 = np.percentile(self.vector, 5)
        self.skewness = st.skew(self.vector)
        self.kurtosis = st.kurtosis(self.vector)
        self.jb_stat = self.size/6 * (self.skewness**2 + 1/4*self.kurtosis**2)
        self.p_value = 1 - st.chi2.cdf(self.jb_stat, df=2)
        self.is_normal = (self.p_value > 0.05) # equivalently jb < 6(self.vector)

    def plot_histrogram (self):
        self.str_tittle += '\n' + 'mean_annual=' + str(np.round(self.mean_annual, self.decimals)) \
           + ' | ' + 'volatility_annual=' + str(np.round(self.volatility_annual, self.decimals)) \
           + '\n' + 'sharpe_ratio=' + str(np.round(self.sharpe_ratio, self.decimals)) \
           + ' | ' + 'var_95=' + str(np.round(self.var_95, self.decimals)) \
           + '\n' + 'skewness=' + str(np.round(self.skewness, self.decimals)) \
           + ' | ' + 'kurtosis=' + str(np.round(self.kurtosis, self.decimals)) \
           + '\n' + 'JB stat=' + str(np.round(self.jb_stat,self.decimals)) \
           + ' | ' + 'p_value=' + str(np.round(self.p_value,self.decimals)) \
           + '\n' + 'is_normal=' + str(self.is_normal) \
           + ' | ' + 'ADF stat=' + str(np.round(self.adf_stat, self.decimals))\
           + '\n ' + 'ADF p_value=' + str(np.round(self.adf_p_value, self.decimals)) \
           + ' | ' + 'is_stationary=' + str(self.is_stationary)
        plt.figure()
        plt.hist(self.vector,bins=100)
        plt.title(self.str_tittle)
        plt.show()

    def fit_arima(self, p=1, q=1):
        """ Ajusta el modelo ARIMA a los datos. """
        # Ajustar el modelo ARIMA
        model = sm.tsa.ARIMA(self.vector, order=(p, 0, q))  # d=0 porque la serie ya es estacionaria
        self.model_fit = model.fit()
        print(self.model_fit.summary())

    def forecast_arima(self, steps=10):
        """ Realiza predicciones con el modelo ARIMA ajustado. """
        if self.model_fit is not None:
            self.forecast = self.model_fit.forecast(steps=steps)
            return self.forecast
        else:
            print("El modelo ARIMA no ha sido ajustado.")
            return None

    def plot_forecast(self, steps=10):
        """ Grafica las predicciones del modelo ARIMA. """
        if self.forecast is not None:
            plt.figure()
            plt.plot(self.vector, label='Datos históricos')
            forecast_index = np.arange(len(self.vector), len(self.vector) + steps)
            plt.plot(forecast_index, self.forecast, label='Predicciones ARIMA', color='red')
            plt.legend()
            plt.title('Predicciones del modelo ARIMA')
            plt.show()
        else:
            print("No hay predicciones disponibles para graficar.")

    def analyze_residuals(self):
        # 1. Obtener los residuos del modelo ARIMA
        residuals = self.model_fit.resid

        # 2. Graficar los residuos
        plt.figure(figsize=(10, 6))
        plt.plot(residuals, label='Residuos')
        plt.axhline(0, linestyle='--', color='r', alpha=0.5)
        plt.title('Residuos del Modelo ARIMA')
        plt.legend()
        plt.show()

        # 3. Graficos ACF y PACF de los residuos
        plt.figure(figsize=(12, 6))

        plt.subplot(121)
        plot_acf(residuals, lags=30, ax=plt.gca())
        plt.title('ACF de los Residuos')

        plt.subplot(122)
        plot_pacf(residuals, lags=30, ax=plt.gca())
        plt.title('PACF de los Residuos')

        plt.tight_layout()
        plt.show()

        # 4. Prueba de Ljung-Box (para autocorrelación)
        ljung_box_results = acorr_ljungbox(residuals, lags=[10], return_df=True)
        print("Resultados del Test de Ljung-Box")
        print(ljung_box_results)

        # 5. Gráfico QQ para verificar la normalidad de los residuos
        sm.qqplot(residuals, line='s')
        plt.title('Gráfico QQ de los Residuos')
        plt.show()

        # 6. Prueba de normalidad Shapiro-Wilk
        shapiro_test = shapiro(residuals)
        print(f'Prueba Shapiro-Wilk: estadístico={shapiro_test[0]}, p-valor={shapiro_test[1]}')

